# Evaluation RoBERTa, previously fine-tuned on MultiNLI datsaset, on  Figurative Language

The datasets used in this notebook where created in the notebook `BERT_Evaluation_on_Figurative_Language.ipynb`


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
! pip install datasets
! pip install -U accelerate
! pip install -U transformers

In [5]:
import torch
import pandas as pd
import numpy as np
import os
import datetime

from transformers import (RobertaTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments,
                          TrainerCallback)
from datasets import (Dataset,
                      load_dataset,
                      load_metric,
                      load_dataset_builder,
                      ClassLabel,
                      Value,
                      Features)

In [6]:
BATCH_SIZE = 32

In [9]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

## Model

In [10]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("an-eve/roberta-base-mnli-2-labels")
model.to(DEVICE)
model.eval()

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [13]:
metric = load_metric('glue', "mnli")

<ipython-input-13-17bf5bf84e68>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## IMPLI (idioms)

In [18]:
full = load_dataset("an-eve/impli", split="train")
full_ne = load_dataset("an-eve/impli_non_entailment", split="train")
full_e = load_dataset("an-eve/impli_entailment", split="train")

In [19]:
tokenized_full = full.map(tokenize_function, batched=True)
tokenized_full_ne = full_ne.map(tokenize_function, batched=True)
tokenized_full_e = full_e.map(tokenize_function, batched=True)

In [20]:
args_test = TrainingArguments("IMPLI",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_full,
    eval_dataset=tokenized_full,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.0106537342071533,
 'eval_accuracy': 0.7042877018424049,
 'eval_runtime': 721.9047,
 'eval_samples_per_second': 32.856,
 'eval_steps_per_second': 1.028}

In [21]:
args_test = TrainingArguments("IMPLI-ne",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_full_ne,
    eval_dataset=tokenized_full_ne,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.1502275466918945,
 'eval_accuracy': 0.40055651252153174,
 'eval_runtime': 230.0183,
 'eval_samples_per_second': 32.81,
 'eval_steps_per_second': 1.026}

In [22]:
args_test = TrainingArguments("IMPLI-e",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_full_e,
    eval_dataset=tokenized_full_e,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.4788479804992676,
 'eval_accuracy': 0.8460301756121692,
 'eval_runtime': 492.7098,
 'eval_samples_per_second': 32.823,
 'eval_steps_per_second': 1.027}

## Idioms

### Adversarial Definitions
Silver non-entailment, replacing an idiom used in a figuartive context with adversarial definition

In [ ]:
adv_ne_dataset = load_dataset("an-eve/adversarial-non-entailment", split='train')
print(adv_ne_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 6035
})


In [ ]:
tokenized_adv_ne_dataset = adv_ne_dataset.map(tokenize_function, batched=True)

In [ ]:
args_test = TrainingArguments("Adversarial-Definitions",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_adv_ne_dataset,
    eval_dataset=tokenized_adv_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.2977490425109863,
 'eval_accuracy': 0.37381938690969346,
 'eval_runtime': 189.28,
 'eval_samples_per_second': 31.884,
 'eval_steps_per_second': 0.999}

### Figurative Context
Silver entailment, replacing an idiom used in a figuartive context with definition

In [ ]:
fig_e_dataset = load_dataset("an-eve/figurative-entailment", split='train')
print(fig_e_dataset)

Generating train split:   0%|          | 0/15641 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 15641
})


In [ ]:
tokenized_fig_e_dataset = fig_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15641 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Figurative-Context",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_fig_e_dataset,
    eval_dataset=tokenized_fig_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.483929306268692,
 'eval_accuracy': 0.8443833514481172,
 'eval_runtime': 499.5437,
 'eval_samples_per_second': 31.311,
 'eval_steps_per_second': 0.979}

### Literal Context
Silver non-entailment, replacing an idiom used in a literal context with definition

In [ ]:
lit_ne_dataset = load_dataset("an-eve/literal-non-entailment", split='train')
print(lit_ne_dataset)

Generating train split:   0%|          | 0/881 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 881
})


In [ ]:
tokenized_lit_ne_dataset = lit_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/881 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Literal-Context",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_lit_ne_dataset,
    eval_dataset=tokenized_lit_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.4607199430465698,
 'eval_accuracy': 0.5073779795686719,
 'eval_runtime': 27.9718,
 'eval_samples_per_second': 31.496,
 'eval_steps_per_second': 1.001}

### Antonyms

Gold non-entailment, manual replacement of idiomatic definition with antonym

In [ ]:
ant_ne_dataset = load_dataset("an-eve/antonym-non-entailment", split='train')
print(ant_ne_dataset)

Generating train split:   0%|          | 0/375 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 375
})


In [ ]:
tokenized_ant_ne_dataset = ant_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Antonym",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_ant_ne_dataset,
    eval_dataset=tokenized_ant_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.0681771039962769,
 'eval_accuracy': 0.64,
 'eval_runtime': 11.6836,
 'eval_samples_per_second': 32.096,
 'eval_steps_per_second': 1.027}

### Definition
Gold entailment, hand written literal definition of idiomatic sentence

In [ ]:
def_e_dataset = load_dataset("an-eve/definition-entailment", split='train')
print(def_e_dataset)

Generating train split:   0%|          | 0/528 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 528
})


In [ ]:
tokenized_def_e_dataset = def_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Definition",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_def_e_dataset,
    eval_dataset=tokenized_def_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.33166754245758057,
 'eval_accuracy': 0.8939393939393939,
 'eval_runtime': 15.8403,
 'eval_samples_per_second': 33.333,
 'eval_steps_per_second': 1.073}

### Non-entailment Hand
Gold non-entailment, hand written non-entailed sentence from idiom

In [ ]:
hand_ne_dataset = load_dataset("an-eve/hand-non-entailment", split='train')
print(hand_ne_dataset)

Generating train split:   0%|          | 0/254 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 254
})


In [ ]:
tokenized_hand_ne_dataset = hand_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/254 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Hand N-E",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_hand_ne_dataset,
    eval_dataset=tokenized_hand_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.6266441345214844,
 'eval_accuracy': 0.31496062992125984,
 'eval_runtime': 7.4865,
 'eval_samples_per_second': 33.928,
 'eval_steps_per_second': 1.069}

## Metaphors

### Paraphrase
Gold entailment, hand written literal paraphrase of metaphors

In [ ]:
man_met_e_dataset = load_dataset("an-eve/man-metaphors-entailment", split='train')
print(man_met_e_dataset)

Generating train split:   0%|          | 0/387 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 387
})


In [ ]:
tokenized_man_met_e_dataset = man_met_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/387 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Metaphors Paraphrase",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_man_met_e_dataset,
    eval_dataset=tokenized_man_met_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.34324657917022705,
 'eval_accuracy': 0.8888888888888888,
 'eval_runtime': 11.0791,
 'eval_samples_per_second': 34.931,
 'eval_steps_per_second': 1.173}

### Non-emtailment Manual
Gold non-entailment, hand written non-entailment of metaphors

In [ ]:
met_ne_dataset = load_dataset("an-eve/metaphors-non-entailment", split='train')
print(met_ne_dataset)

Generating train split:   0%|          | 0/281 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 281
})


In [ ]:
tokenized_met_ne_dataset = met_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/281 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("N-E Metaphors Paraphrase",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_met_ne_dataset,
    eval_dataset=tokenized_met_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.6286956667900085,
 'eval_accuracy': 0.7829181494661922,
 'eval_runtime': 8.0013,
 'eval_samples_per_second': 35.119,
 'eval_steps_per_second': 1.125}

### Automatic Metaphor
Silver entailment, automatic replacement of metaphoric expressions with literal

In [ ]:
aut_e_dataset = load_dataset("an-eve/automatic-metaphors-entailment", split='train')
print(aut_e_dataset)

Generating train split:   0%|          | 0/644 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 644
})


In [ ]:
tokenized_aut_e_dataset = aut_e_dataset.map(tokenize_function, batched=True)

In [ ]:
args_test = TrainingArguments("Automatic-Metaphors",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_aut_e_dataset,
    eval_dataset=tokenized_aut_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.1110224649310112,
 'eval_accuracy': 0.9720496894409938,
 'eval_runtime': 18.3835,
 'eval_samples_per_second': 35.031,
 'eval_steps_per_second': 1.142}